# SHapley Additive exPlanations (SHAP)

## Library Imports 

In [ ]:
import os
import pickle

import pandas as pd
import shap

In [ ]:
RAND_STATE = 0

## Importing the Train and Test Sets

In [ ]:
dataset_folder = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/datasets"

In [ ]:
X_train = pd.read_csv(os.path.join(dataset_folder, "obesity_X_train.csv"), index_col=0)

In [ ]:
X_test = pd.read_csv(os.path.join(dataset_folder, "obesity_X_test.csv"), index_col=0)

As with the rest of the stages, we make a variant of the sets without the height and weight columns for comparison:

In [ ]:
X_train_no_hw = X_train.drop(["Height", "Weight"], axis=1)

In [ ]:
X_test_no_hw = X_test.drop(["Height", "Weight"], axis=1)

## Importing the Random Forest Classifiers
We import the random forest classifiers that we trained previously. SHAP values will be calculated for these models to explain their predictions.

In [ ]:
def import_model(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/models/{filename}"
    with open(file_path, 'rb') as file: 
        model = pickle.load(file)
    print(f"Model imported from {file_path}")
    return model

In [ ]:
rand_forest = import_model("rand_forest.pkl")

In [ ]:
rand_forest_no_hw = import_model("rand_forest_no_hw.pkl")

## Importing the Target Feature Label Encoder
We import the label encoder for the target feature so that we can encode the original values for indexing purposes.

In [ ]:
def import_encoder(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/encoders/{filename}"
    with open(file_path, 'rb') as file: 
        encoder = pickle.load(file)
    print(f"Encoder imported from {file_path}")
    return encoder

In [ ]:
target_le = import_encoder("target_le.pkl")

In [ ]:
target_class_label_d = {cls: idx for idx, cls in enumerate(target_le.classes_)}
target_class_label_d

## Explanations

First, we create the explainers and calculate the SHAP values:

In [ ]:
explainer = shap.TreeExplainer(rand_forest)
shap_values = explainer(X_test)

In [ ]:
explainer_no_hw = shap.TreeExplainer(rand_forest_no_hw)
shap_values_no_hw = explainer_no_hw(X_test_no_hw)

### With Height and Weight
Now, we can plot beeswarm plots for the model trained with the height and weight using the SHAP values.
For the obese class:

In [ ]:
shap.plots.beeswarm(shap_values[:, :, target_class_label_d["Yes"]], max_display=X_test.shape[1])

For the non-obese class:

In [ ]:
shap.plots.beeswarm(shap_values[:, :, target_class_label_d["No"]], max_display=X_test.shape[1])

### Without Height and Weight
Similarly, we plot beeswarm plots for the model trained without the height and weight.
For the obese class:

In [ ]:
shap.plots.beeswarm(shap_values_no_hw[:, :, target_class_label_d["Yes"]], max_display=X_test_no_hw.shape[1])

For the non-obese class:

In [ ]:
shap.plots.beeswarm(shap_values_no_hw[:, :, target_class_label_d["No"]], max_display=X_test_no_hw.shape[1])